# Intro. to Snorkel: Extracting Spouse Relations from the News

## Part III: Training an End Extraction Model

In this final section of the tutorial, we'll use the noisy training labels we generated in the last tutorial part to train our end extraction model.

For this tutorial, we will be training a Bi-LSTM, a state-of-the-art deep neural network implemented in [TensorFlow](https://www.tensorflow.org/).

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

# TO USE A DATABASE OTHER THAN SQLITE, USE THIS LINE
# Note that this is necessary for parallel execution amongst other things...
# os.environ['SNORKELDB'] = 'postgres:///snorkel-intro'

from snorkel import SnorkelSession
session = SnorkelSession()

We repeat our definition of the `Spouse` `Candidate` subclass:

In [2]:
from snorkel.models import candidate_subclass

Indicator = candidate_subclass('Indicator', ['tr', 'ind'])

Created a table in database


We reload the probabilistic training labels:

In [3]:
from snorkel.annotations import load_marginals

train_marginals = load_marginals(session, split=0)

We also reload the candidates:

In [4]:
train_cands = session.query(Indicator).filter(Indicator.split == 0).order_by(Indicator.id).all()
dev_cands   = session.query(Indicator).filter(Indicator.split == 1).order_by(Indicator.id).all()
test_cands  = session.query(Indicator).filter(Indicator.split == 2).order_by(Indicator.id).all()
d_cands = session.query(Indicator).filter(Indicator.split == 3).order_by(Indicator.id).all()
print(len(d_cands))

39991


Finally, we load gold labels for evaluation:

In [5]:
from snorkel.annotations import load_gold_labels

L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)

Now we can setup our discriminative model. Here we specify the model and learning hyperparameters.

They can also be set automatically using a search based on the dev set with a [GridSearch](https://github.com/HazyResearch/snorkel/blob/master/snorkel/learning/utils.py) object.

In [6]:
from snorkel.learning.pytorch import LSTM

train_kwargs = {
    'lr':            0.01,
    'embedding_dim': 50,
    'hidden_dim':    50,
    'n_epochs':      10,
    'dropout':       0.25,
    'seed':          1701
}

lstm = LSTM(n_threads=None)
lstm.train(train_cands, train_marginals, X_dev=dev_cands, Y_dev=L_gold_dev, **train_kwargs)

[LSTM] Training model
[LSTM] n_train=4412  #epochs=10  batch size=64


/home/pegah/anaconda3/envs/snorkel/lib/python3.6/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[LSTM] Epoch 1 (7.74s)	Average loss=0.613073	Dev F1=0.00
[LSTM] Epoch 2 (15.77s)	Average loss=0.586777	Dev F1=0.00
[LSTM] Epoch 3 (23.58s)	Average loss=0.476237	Dev F1=11.76
[LSTM] Epoch 4 (31.63s)	Average loss=0.338215	Dev F1=11.94
[LSTM] Epoch 5 (39.63s)	Average loss=0.299423	Dev F1=10.86
[LSTM] Epoch 6 (47.40s)	Average loss=0.275878	Dev F1=11.80
[LSTM] Epoch 7 (55.34s)	Average loss=0.264643	Dev F1=11.73
[LSTM] Epoch 8 (64.65s)	Average loss=0.240746	Dev F1=12.96
[LSTM] Epoch 9 (73.00s)	Average loss=0.223185	Dev F1=12.66
[LSTM] Model saved as <LSTM>
[LSTM] Epoch 10 (80.61s)	Average loss=0.218958	Dev F1=12.27
[LSTM] Training done (81.06s)
[LSTM] Loaded model <LSTM>


Now, we get the precision, recall, and F1 score from the discriminative model:

In [7]:
p, r, f1 = lstm.score(test_cands, L_gold_test)
print("Prec: {0:.3f}, Recall: {1:.3f}, F1 Score: {2:.3f}".format(p, r, f1))

Prec: 0.000, Recall: 0.000, F1 Score: 0.000


We can also get the candidates returned in sets (true positives, false positives, true negatives, false negatives) as well as a more detailed score report:

In [8]:
tp, fp, tn, fn = lstm.error_analysis(session, test_cands, L_gold_test)

Scores (Un-adjusted)
Pos. class accuracy: 0.0
Neg. class accuracy: 0.533
Precision            0.0
Recall               0.0
F1                   0.0
----------------------------------------
TP: 0 | FP: 28 | TN: 32 | FN: 3



Note that if this is the final test set that you will be reporting final numbers on, to avoid biasing results you should not inspect results.  However you can run the model on your _development set_ and, as we did in the previous part with the generative labeling function model, inspect examples to do error analysis.

You can also improve performance substantially by increasing the number of training epochs!

Finally, we can save the predictions of the model on the test set back to the database. (This also works for other candidate sets, such as unlabeled candidates.)

In [11]:
lstm.save_marginals(session, test_cands)
tmp_cands = session.query(Indicator).filter(Indicator.split == 3).order_by(Indicator.id).all()
print(len(tmp_cands))
# print(lstm.predictions(tmp_cands))
preds = lstm.predictions(tmp_cands)
from snorkel.viewer import SentenceNgramViewer

extractions = []
for i, p in enumerate(preds):
    if p == 1:
        extractions.append(tmp_cands[i])
# write them into a file
print(len(extractions))
for i in extractions:
    trend_id = i.tr_id
    indicator_id = ind_id
    
    print(i.get_contexts())
    


sv2 = SentenceNgramViewer(extractions, session)
# sv2

Saved 63 marginals
39991


RuntimeError: $ Torch: not enough memory: you tried to allocate 0GB. Buy new RAM! at /opt/conda/conda-bld/pytorch-cpu_1544218188686/work/aten/src/TH/THGeneral.cpp:201

##### More importantly, you completed the introduction to Snorkel! Give yourself a pat on the back!